In [3]:
import findspark
findspark.init()
# findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'C:\\Spark\\spark-hadoop'

In [4]:
import pyspark
from pyspark.sql import SparkSession


# spark = SparkSession.builder.master("yarn")\
spark = SparkSession.builder.master("local[*]")\
.appName("ch26Classification").getOrCreate()


#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "1g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "4")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [5]:
sc = spark.sparkContext

In [6]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

In [7]:
data_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/"
# data_dir = "/user/kranthidr/dataSets/spark-guide/"
path = data_dir + "binary-classification"

In [8]:
bInput = spark.read.format("parquet").load(path)\
  .selectExpr("features", "cast(label as double) as label")

In [9]:
bInput.show(10,False)

+--------------+-----+
|features      |label|
+--------------+-----+
|[3.0,10.1,3.0]|1.0  |
|[1.0,0.1,-1.0]|0.0  |
|[1.0,0.1,-1.0]|0.0  |
|[2.0,1.1,1.0] |1.0  |
|[2.0,1.1,1.0] |1.0  |
+--------------+-----+



In [10]:
# COMMAND ----------
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

# _init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#         maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-6, fitIntercept=True,                  
#         threshold=0.5, thresholds=None, probabilityCol="probability",                  
#         rawPredictionCol="rawPrediction", standardization=True, weightCol=None,                  
#         aggregationDepth=2, family="auto",                  
#         lowerBoundsOnCoefficients=None, upperBoundsOnCoefficients=None,                  
#         lowerBoundsOnIntercepts=None, upperBoundsOnIntercepts=None):

In [11]:
print(lr.explainParams()) # see all parameters

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [12]:
lrModel = lr.fit(bInput)

In [13]:
# COMMAND ----------
print(lrModel.coefficients)
print(lrModel.intercept)

[6.848741326855034,0.35356589010197487,14.814900276915889]
-10.225695864480993


In [14]:
# COMMAND ----------
summary = lrModel.summary

In [15]:
print(summary.areaUnderROC)

1.0


In [16]:
summary.roc.show()

+---+------------------+
|FPR|               TPR|
+---+------------------+
|0.0|               0.0|
|0.0|0.3333333333333333|
|0.0|               1.0|
|1.0|               1.0|
|1.0|               1.0|
+---+------------------+



In [17]:
summary.pr.show()

+------------------+---------+
|            recall|precision|
+------------------+---------+
|               0.0|      1.0|
|0.3333333333333333|      1.0|
|               1.0|      1.0|
|               1.0|      0.6|
+------------------+---------+



In [18]:
# COMMAND ----------

summary.objectiveHistory

[0.6730116670092565,
 0.5042829330409728,
 0.36356862066874396,
 0.1252407018038338,
 0.08532556611276212,
 0.03550487641573043,
 0.01819649450857124,
 0.008817369922959128,
 0.004413673785392138,
 0.002194038351234706,
 0.001096564114808084,
 0.0005476575519853136,
 0.000273762379514901,
 0.0001368465223657475,
 6.84180903707058e-05,
 3.4207077910384856e-05,
 1.710317666423191e-05,
 8.551470106426885e-06,
 4.275703677941412e-06,
 2.1378240117781396e-06,
 1.0688564054651744e-06,
 5.342600202575258e-07,
 2.668135105897087e-07,
 1.32046278653136e-07,
 6.768401481681801e-08,
 3.3145477184834547e-08,
 1.6151438837488498e-08,
 8.309350118268437e-09]

In [19]:
# COMMAND ----------

from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier()

# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                          probabilityCol="probability", rawPredictionCol="rawPrediction",                  
#                          maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                          maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity="gini",                  
#                          seed=None)

In [20]:
print(dt.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree.

In [21]:
dtModel = dt.fit(bInput)

In [22]:
# COMMAND ----------
from pyspark.ml.classification import RandomForestClassifier
rfClassifier = RandomForestClassifier()


# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                          probabilityCol="probability", rawPredictionCol="rawPrediction",                  
#                          maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                          maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10, impurity="gini",                  
#                          numTrees=20, featureSubsetStrategy="auto", seed=None, subsamplingRate=1.0)

In [23]:
print(rfClassifier.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: entropy, gini (default: gini)
labelCol: label column name. (default: label)
maxBins: Max

In [24]:
trainedModel = rfClassifier.fit(bInput)

In [25]:
# COMMAND ----------
from pyspark.ml.classification import GBTClassifier
gbtClassifier = GBTClassifier()

# init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                         maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                         maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10,                  
#                         lossType="logistic", maxIter=20, stepSize=0.1, seed=None, subsamplingRate=1.0)


In [26]:
print(gbtClassifier.explainParams())

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: logistic (default: logistic)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. (>= 0)

In [27]:
trainedModel = gbtClassifier.fit(bInput)

In [28]:
# COMMAND ----------
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  p
#                          robabilityCol="probability", rawPredictionCol="rawPrediction", smoothing=1.0,                 
#                          modelType="multinomial", thresholds=None, weightCol=None)

In [29]:
print(nb.explainParams())

featuresCol: features column name. (default: features)
labelCol: label column name. (default: label)
modelType: The model type which is a string (case-sensitive). Supported options: multinomial (default) and bernoulli. (default: multinomial)
predictionCol: prediction column name. (default: prediction)
probabilityCol: Column name for predicted class conditional probabilities. Note: Not all models output well-calibrated probability estimates! These probabilities should be treated as confidences, not precise probabilities. (default: probability)
rawPredictionCol: raw prediction (a.k.a. confidence) column name. (default: rawPrediction)
smoothing: The smoothing parameter, should be >= 0, default is 1.0 (default: 1.0)
thresholds: Thresholds in multi-class classification to adjust the probability of predicting each class. Array must have length equal to the number of classes, with values > 0, excepting that at most one value may be 0. The class with largest value p/t is predicted, where p is 

In [30]:
trainedModel = nb.fit(bInput.where("label != 0"))

In [31]:
# COMMAND ----------
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [32]:
model = trainedModel

In [33]:
out = model.transform(bInput)\
  .select("prediction", "label")\
  .rdd.map(lambda x: (float(x[0]), float(x[1])))

In [34]:
metrics = BinaryClassificationMetrics(out)

In [35]:
# COMMAND ----------
# print(metrics.areaUnderPR)
# print(metrics.areaUnderROC)
# print("Receiver Operating Characteristic")

#Eror in pyspark-lab kernel
# Py4JJavaError: An error occurred while calling o438.areaUnderPR.
# : org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 236.0 failed 4 times, 
#             most recent failure: Lost task 0.3 in stage 236.0 (TID 415, wn05.itversity.com, executor 3): 
#                     java.io.IOException: 
#                         Cannot run program "/home/kranthidr/.virtualenvs/pyspark-lab/bin/python3.5": error=2,
#                             No such file or directory

In [36]:
# metrics.roc.toDF().show()
# #AttributeError: 'BinaryClassificationMetrics' object has no attribute 'roc'

In [37]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [38]:
bceval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC')

# Init signature: BinaryClassificationEvaluator(
#rawPredictionCol='rawPrediction', labelCol='label', metricName='areaUnderROC')
# Docstring:     
# .. note:: Experimental

In [39]:
transed = trainedModel.transform(bInput)

In [40]:
transed.columns

['features', 'label', 'rawPrediction', 'probability', 'prediction']

In [42]:
#ml_bmetrics = bceval.evaluate(transed)

# # y4JJavaError: An error occurred while calling o772.evaluate.
# # : org.apache.spark.SparkException: Job aborted due to stage failure:
# #             Task 0 in stage 483.0 failed 4 times, most recent failure: 
# #                 Lost task 0.3 in stage 483.0 (TID 864, wn03.itversity.com, executor 2): 
# #                     java.lang.ArrayIndexOutOfBoundsException: 1
# # 	at org.apache.spark.ml.linalg.DenseVector.apply(Vectors.scala:462)

In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [44]:
mceval = MulticlassClassificationEvaluator()

# Init signature: MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
# Docstring:   

In [45]:
transed = trainedModel.transform(bInput)

In [46]:
transed.columns

['features', 'label', 'rawPrediction', 'probability', 'prediction']

In [47]:
ml_bmetrics = mceval.evaluate(transed)

In [48]:
ml_bmetrics.real

0.2285714285714286